# Exploring Data using Python and SQLite
In this notebook I demonstrate the use of Python and SQL to manipulate datasets.
I will be using SQLite databases, which do not require a live SQL server. 

While SQL implementation can vary, there can be many shared keywords and operators that make learning one type, like SQLite, beneficial as knowledge can be carried over.

In [ ]:
import pandas as pd
import sqlite3

## Datasets Used

The [sf-salaries dataset](https://www.kaggle.com/kaggle/sf-salaries) is provided by Kaggle and has a CC0: Public Domain licence.

The datset is taken from [Transparent California](https://transparentcalifornia.com/pages/about/) which is managed by the [Nevada Policy Research Institute](https://www.npri.org/about/), an IRS 501(c) (3) organization.

The SQLitetutorial-example-database is provided by sqlitetutorial.net and can be found [here](https://www.sqlitetutorial.net/sqlite-sample-database/) as the chinook.db.

We first need to connect to the sf-salaries database. We can store the database connection in an object.

We should remember to close connections once we are done with them. For SQLite, that command is conn.close().

In [ ]:
conn = sqlite3.connect("""../input/sf-salaries/database.sqlite""")

# SELECT, FROM, SELECT *, and LIMIT
Using pandas and the method read_sql, we can input the query and the database connection object, conn.

We can treat the data as a dataframe and use pandas functions to look at the data, or use purely SQL queries. 

The basic query uses SELECT AND FROM; SELECT for choosing columns, and FROM for choosing a table. Using SELECT * returns all columns from a table.

Adding LIMIT to the end of every query ensures the query doesn't return too many rows.

In [ ]:
df = pd.read_sql("SELECT * FROM salaries LIMIT 10;",conn)
df.head(10)

In [ ]:
df.info()

Using OFFSET, we can view rows starting from the offset row.

In [ ]:
pd.read_sql("SELECT * FROM salaries LIMIT 10 OFFSET 5;",conn)

### COUNT, DISTINCT, AS
Using COUNT(*) gives us the total number of rows in the table.

Using DISTINCT with the column Year gives us each year.



In [ ]:
pd.read_sql("SELECT COUNT(*) FROM salaries LIMIT 10;",conn)

In [ ]:
pd.read_sql("SELECT DISTINCT Year FROM salaries;",conn)

## Subquery
Although a count of distinct EmployeeName appears to be significantly different from the total number of rows, a count of a subquery of DISTINCT rows shows no duplicate rows.

Some people appear to have the same first and last names. Without employee IDs, we can't be sure these are different people, or the same people having changed jobs.

We can use the AS keyword to display a more readable column name.

In [ ]:
pd.read_sql("SELECT COUNT(DISTINCT EmployeeName) AS rows FROM salaries LIMIT 10;",conn)

In [ ]:
pd.read_sql("""SELECT COUNT(*) FROM (SELECT DISTINCT * FROM salaries) LIMIT 10;""",conn)

We can store the query in its own object and pass the object into the pd.read_sql() method. This makes the code reusableand a little more readable. 

The only agency is San Francisco and there does not appear to be any values under the Notes column, so we can ignore these columns.

In [ ]:
query_agency = """
SELECT DISTINCT agency
FROM
    salaries
LIMIT 10;"""

pd.read_sql(query_agency, conn)

In [ ]:
query_notes = """
SELECT DISTINCT notes
FROM
    salaries
LIMIT 10;"""

pd.read_sql(query_notes, conn)

A count of distinct job titles returns 2159 jobs.

In [ ]:
query_distinct_jobs = """
SELECT 
    COUNT(DISTINCT JobTitle) AS Unique_Jobs
FROM
    salaries
LIMIT 10;
"""
pd.read_sql(query_distinct_jobs, conn)

# WHERE, GROUP BY, ORDER BY, and HAVING
WHERE allows us to include conditional statements that must return true for a row to be returned.

Here, we want TotalPayBenefits to not be NULL, or to equal the integer 0. We use the AND keyword to chain conditional statements.

Using some aggregation functions and GROUP BY, we can group together shared values such as JobTitle, and return a count using COUNT. Or we can get the average, min, and max values using AVG(), MIN(), MAX().

With ORDER BY and DESC, we can return a query according to a column, in descending order. 

Below is displayed the top 20 highest paid jobs.

In [ ]:
query_avg_totalpaybenefits = """
SELECT 
    JobTitle, COUNT(JobTitle) Job_Count, AVG(TotalPayBenefits) AVG_TOT_PAY_BENE, AVG(TotalPay) AVG_TOT_PAY, MAX(TotalPayBenefits) MAX_TOT_PAY_BENE, MIN(TotalPayBenefits) MIN_TOT_PAY_BENE
FROM
    salaries
WHERE
    TotalPayBenefits IS NOT NULL AND TotalPayBenefits != 0
GROUP BY
    JobTitle
ORDER BY
    AVG(TotalPayBenefits) DESC
LIMIT 20;
"""
pd.read_sql(query_avg_totalpaybenefits, conn)

Here are the top 20 most popular jobs. 

In [ ]:
query_count_jobs = """
SELECT 
    JobTitle, COUNT(JobTitle) Job_Count, AVG(TotalPayBenefits) AVG_TOT_PAY_BENE, AVG(TotalPay) AVG_TOT_PAY
FROM
    salaries
WHERE
    TotalPayBenefits IS NOT NULL AND TotalPayBenefits != 0
GROUP BY
    JobTitle
ORDER BY
    COUNT(JobTitle) DESC
LIMIT 20;
"""
pd.read_sql(query_count_jobs, conn)

HAVING is used to filter the results of aggregate functions.

In [ ]:
query_count_jobs_two = """
SELECT 
    JobTitle, COUNT(JobTitle) Job_Count, AVG(TotalPayBenefits) AVG_TOT_PAY_BENE, AVG(TotalPay) AVG_TOT_PAY
FROM
    salaries
WHERE
    TotalPayBenefits IS NOT NULL AND TotalPayBenefits != 0
GROUP BY
    JobTitle
HAVING
    COUNT(JobTitle) BETWEEN 500 AND 1000
ORDER BY
    COUNT(JobTitle) DESC
LIMIT 20;
"""
pd.read_sql(query_count_jobs_two, conn)

### LIKE and wildcards %
We can use the keyword LIKE coupled with wildcards '%' to match all characters before and after a given value.



This dataset is rather incomplete. Many of these jobs aren't organized into departments or common fields. While some jobs have a department in their job title, such as those below, this won't catch all law enforcement jobs, such as SHERIFF'S SERGEANT, or DEPUTY SHERIFF.

In [ ]:
query_pol_depart = """
SELECT jobtitle, 
    COUNT(jobtitle) AS Count
FROM 
    salaries 
WHERE 
    jobtitle LIKE '%police department%' AND
    Year = 2011
GROUP BY 
    jobtitle 
ORDER BY 
    Count DESC
LIMIT 5;"""
pd.read_sql(query_pol_depart, conn)

In [ ]:
query_depart = """
SELECT jobtitle, 
    COUNT(jobtitle) AS Count
FROM 
    salaries 
WHERE 
    jobtitle LIKE 'department%' AND
    Year = 2011
GROUP BY 
    jobtitle 
ORDER BY 
    Count DESC
LIMIT 10;"""
pd.read_sql(query_depart, conn)

In [ ]:
query_depart_two = """
SELECT jobtitle, 
    COUNT(jobtitle) AS Count
FROM 
    salaries 
WHERE 
    jobtitle LIKE '%department' AND
    Year = 2011
GROUP BY 
    jobtitle 
ORDER BY 
    Count DESC
LIMIT 10;"""
pd.read_sql(query_depart_two, conn)

In [ ]:
query_serge = """
SELECT jobtitle, 
    COUNT(jobtitle) AS Count
FROM 
    salaries 
WHERE 
    jobtitle LIKE '%sergeant%' AND
    Year = 2011
GROUP BY 
    jobtitle 
ORDER BY 
    Count DESC
LIMIT 10;"""
pd.read_sql(query_serge, conn)

In [ ]:
query_deputy = """
SELECT jobtitle, 
    COUNT(jobtitle) AS Count
FROM 
    salaries 
WHERE 
    jobtitle LIKE '%deputy%' AND
    Year = 2011
GROUP BY 
    jobtitle 
ORDER BY 
    Count DESC
LIMIT 5;"""
pd.read_sql(query_deputy, conn)

# UNION and INTERSECT
Using UNION, we can append rows of queries with matching columns and datatypes.

In [ ]:
query_union = """
SELECT jobtitle, 
    COUNT(jobtitle) AS Count
FROM 
    salaries 
WHERE 
    jobtitle LIKE '%officer' AND
    Year = 2011
GROUP BY 
    jobtitle
                        
                        
UNION
                 
                 
SELECT jobtitle, 
    COUNT(jobtitle) AS Count
FROM 
    salaries 
WHERE 
    jobtitle LIKE 'police%' AND
    Year = 2011
GROUP BY 
    jobtitle 
ORDER BY 
    Count DESC
LIMIT 10;"""
pd.read_sql(query_union, conn)

INTERSECT returns the matching rows found in both queries.

In [ ]:
query_intersect_union = """
SELECT jobtitle, 
    COUNT(jobtitle) AS Count
FROM 
    salaries 
WHERE 
    jobtitle LIKE '%officer%' AND
    Year = 2011
GROUP BY 
    jobtitle
                        
                        
INTERSECT
                 
                
SELECT jobtitle, 
    COUNT(jobtitle) AS Count
FROM 
    salaries 
WHERE 
    jobtitle LIKE '%police%' AND
    Year = 2011
GROUP BY 
    jobtitle 
ORDER BY 
    Count DESC
LIMIT 20;"""
pd.read_sql(query_intersect_union, conn)

# INNER JOIN, LEFT JOIN, RIGHT JOIN, FULL OUTER JOIN
Let's close the sf-salaries database connection and take a look at the SQLitetutorial-example-database, or the chinook, database.

In [ ]:
conn.close()

In [ ]:
chinook_conn = sqlite3.connect("""../input/sqlitetutorial-example-database/chinook.db""")

We can list the tables in the SQLite database by querying the sqlite_master table.

In [ ]:
query_chinook = """
SELECT type, name, tbl_name
FROM
    sqlite_master
WHERE
    type = "table"
LIMIT 25;"""
pd.read_sql(query_chinook, chinook_conn)

If we take a look at the albums and tracks tables, we see the AlbumID column in both.

In [ ]:
chinook_albums = """
SELECT *
FROM
    albums
LIMIT 5;"""
pd.read_sql(chinook_albums, chinook_conn)

In [ ]:
chinook_tracks = """
SELECT *
FROM
    tracks
LIMIT 5;"""
pd.read_sql(chinook_tracks, chinook_conn)

We can JOIN these tables using AlbumId as a key.
With INNER JOIN (or just JOIN) and ON, we need to refer to the common key in the format *table1.column1 = table2.column2*.

In [ ]:
join_count_on = """
SELECT title album_title, COUNT(DISTINCT name) track_count  
FROM
    tracks
INNER JOIN
    albums ON tracks.AlbumId = albums.AlbumId
GROUP BY
    title
ORDER BY trackid
LIMIT 10;"""
pd.read_sql(join_count_on, chinook_conn)

If the column key shares the same name across both tables, we can use USING(*common_column*).

In [ ]:
join_count_using = """
SELECT title album_title, COUNT(DISTINCT name) track_count  
FROM
    tracks
JOIN
    albums USING(AlbumId)
GROUP BY
    title
ORDER BY trackid
LIMIT 10;"""
pd.read_sql(join_count_using, chinook_conn)

A LEFT JOIN includes all the rows of the first table, regardless of a matching row in the second table.

Only the Sales Support Agent employees 3, 4, and 5, having matching SupportRepIds in the customers table, so all other employees will have NULL customer matches.

In [ ]:
left_join = """
SELECT EmployeeId, CustomerId
FROM
    employees
LEFT JOIN 
    customers ON employees.EmployeeId = customers.SupportRepId
WHERE 
    EmployeeId GLOB '[4578]'
ORDER BY
    EmployeeId DESC
LIMIT 10;"""
pd.read_sql(left_join, chinook_conn)

SQLite does not explicitly support RIGHT JOIN or FULL OUTER JOIN.

A RIGHT JOIN can be indirectly used by switching the positions of the tables joined.

The query below reverses the table positions of the query above.

As every CustomerId has a SalesRepId(or EmployeeId), no nulls are listed under EmployeeId.

In [ ]:
reverse_left_join = """
SELECT CustomerId, EmployeeId
FROM
    customers 
LEFT JOIN 
    employees ON employees.EmployeeId = customers.SupportRepId
ORDER BY
    EmployeeId DESC
LIMIT 10;"""
pd.read_sql(reverse_left_join, chinook_conn)

A FULL OUTER JOIN can also be indirectly used, by using UNION ALL between two queries with the second query reversing its table positions.

In [ ]:
full_outer_join = """
SELECT
    EmployeeId, CustomerId
FROM
    employees
LEFT JOIN 
    customers ON employees.EmployeeId = customers.SupportRepId
    
UNION ALL

SELECT 
    EmployeeId, CustomerId
FROM
    customers 
LEFT JOIN 
    employees ON customers.SupportRepId = employees.EmployeeId
ORDER BY
    CustomerId ASC
LIMIT 10;
"""
pd.read_sql(full_outer_join, chinook_conn)

# CREATE, INSERT, UPDATE, DELETE, DROP
With Kaggle, input files are read-only. To utilize keywords that modify a table such as INSERT or UPDATE, an output file needs to be created and connected to.

In [ ]:
chinook_conn.close()

kaggle's version of pandas doesn't support to_sql, for writing to tables.

 

In [ ]:
'to_sql' in dir(pd)

We can instead create a function to use the SQLite3 module to handle queries.

Our function exe_query(), takes two strings, the first being the name of the database, and the second being the query.

Using the context manager *with*, we can have the database connection close itself once it finishes running.

In [ ]:
def exe_query(database, query):
    with sqlite3.connect(database) as conn:
        c = conn.cursor()
        c.execute(query)
        conn.commit()

Let us create a table *students*, and then INSERT some VALUES.

In [ ]:
query_create_table ="""
CREATE TABLE IF NOT EXISTS students (
student_id INTEGER PRIMARY KEY,
first_name TEXT,
last_name TEXT
);"""
exe_query('output.db', query_create_table)

In [ ]:
query_insert_students = """
INSERT INTO students (student_id, first_name, last_name)
VALUES 
    (1, 'Jacob', 'Langley'),
    (2, 'Anna', 'Robertson'),
    (3, 'Wang', 'Li');"""
exe_query('output.db', query_insert_students)

We'll continue to use pandas just to make the output more readable. We'll open a connection to the database to do so.

In [ ]:
output_conn = sqlite3.connect('output.db')

In [ ]:
show_table = """
SELECT *
FROM students
LIMIT 10;
"""
pd.read_sql(show_table, output_conn)

Using UPDATE we can change a column's values.

In [ ]:
update_jacob_jake = """
UPDATE students
SET first_name = 'Jake'
WHERE
    student_id = 1;
"""
exe_query('output.db', update_jacob_jake)

In [ ]:
pd.read_sql(show_table, output_conn)

INSERT allows us to insert additional VALUES.

In [ ]:
insert_more = """
INSERT INTO students(student_id, first_name, last_name)
VALUES 
    (4, 'Ando', 'Kobayashi'),
    (5, 'Johan', 'Meyer');"""
exe_query('output.db', insert_more)

In [ ]:
pd.read_sql(show_table, output_conn)

And DELETE allows us to delete them from the table.

In [ ]:
delete_anna = """
DELETE FROM students
WHERE 
    student_id = 2 AND first_name = 'Anna';"""
exe_query('output.db', delete_anna)

In [ ]:
pd.read_sql(show_table, output_conn)

Finally, we can also delete, or DROP, tables no longer in use. Under sqlite_master, we can see the students table.

In [ ]:
pd.read_sql('SELECT * FROM sqlite_master', output_conn)

In [ ]:
drop_table = """
DROP TABLE students;"""
exe_query('output.db', drop_table)

After the DROP query is executed, checking sqlite_master shows the table is no longer listed.

In [ ]:
pd.read_sql('SELECT * FROM sqlite_master', output_conn)

In [ ]:
output_conn.close()

This concludes a basic overview of using Python and SQLite to explore data.